In [2]:
import sweetviz as sv
import pandas as pd

In [8]:
DATA_PATH = 'data/'

In [12]:
encuesta_df = pd.read_csv(f"{DATA_PATH}datosEncuesta2206.csv")
sensor_df = pd.read_csv(f"{DATA_PATH}datosSensorTurismo2206.csv")
ssids_df  = pd.read_csv(f"{DATA_PATH}ssidsAlcoi.csv")

En el conjunto de datos datos SSIDS Alcoi tenemos todas las direcciones de los puntos de conexión de la ciudad de Alocai.

En el dataset datosSensorTurismo2206 tenemos 




De cada dispositivo, si la red a la que más se conecta no es de Alcoi es un turista.


De este criterio saldrá un listado de MAC (dispositivos) que visitantes.


In [4]:

#analyzing the dataset
advert_report = sv.analyze(encuesta_df)
#display the report
advert_report.show_html('Advertising.html')

                                             |                              | [  0%]   00:00 -> (? left)

/home/luis/anaconda3/envs/TFM/lib/python3.9/site-packages/sweetviz/utils.py:34: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clamped_series = clamped_series.append(other_series, ignore_index=False)
/home/luis/anaconda3/envs/TFM/lib/python3.9/site-packages/sweetviz/graph_associations.py:223: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_dataframe = new_dataframe.append(pd.Series(empty_row_dict, name=feature))
/home/luis/anaconda3/envs/TFM/lib/python3.9/site-packages/sweetviz/graph_associations.py:223: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_dataframe = new_dataframe.append(pd.Series(empty_row_dict, name=feature))
/home/luis/anaconda3/envs/TFM/lib/python3.9/site-packages/sweetviz/graph_associations.py:223: Fut

Report Advertising.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [58]:
# advert_report = sv.analyze(sensor_df)
#display the report
# advert_report.show_html('sensor_df.html')


sorted_mac_ssid = sensor_df.groupby(by=["mac", 'ssid']).count().sort_values('date', ascending = False)
sorted_mac_ssid.value_counts()
# sorted_mac_ssid
# ssids_df[ssids_df['ssid'] == 'cca78a1e963f3ba3443bf3e54a9aa939f73d223d']

date 
1        17753
2          559
3          202
4          128
5           43
         ...  
59           1
65           1
71           1
73           1
12807        1
Length: 67, dtype: int64

In [56]:
ssids_df = ssids_df.drop_duplicates()
sorted_mac = sensor_df.groupby(by=["mac", "ssid"]).count().sort_values('date', ascending = False)
sorted_mac

,,date
mac,ssid,
d29675014f618f3ef3f6b75f146b4be71c513c16,cca78a1e963f3ba3443bf3e54a9aa939f73d223d,12807
5cbcffeab5fa9206a8c2712ef085daab688dd086,cca78a1e963f3ba3443bf3e54a9aa939f73d223d,1267
273a50bed34ca35e5d1987ea3b05ce765aae0e0e,917c1e46b3861300aebf9a14e47a6ec989c7ac9e,933
6ec65bc349733bc0e85d17294d3a9cb6d8b8bdc8,ffd2edd79fe4bac98610a2ad6892b53395e44e34,724
de6157c0185924febf402efa9e4c36a4cbe4cbb1,cca78a1e963f3ba3443bf3e54a9aa939f73d223d,496
...,...,...
57502713dc0b287ab0e893c8313f55598077630f,7a592c4a3d20f50b2ba09a85b67c6d2fc925664b,1
5751a0c6507a8accd8a13005b6976d663db69bad,7a592c4a3d20f50b2ba09a85b67c6d2fc925664b,1
575588aec3617b91ea18facfd1e52a939b47dd6f,b67ede98966e4c418becb185d7efc74e2f59aa15,1


In [24]:
ssids_df
advert_report = sv.analyze(ssids_df)
#display the report
advert_report.show_html('ssids_df.html')

                                             |                              | [  0%]   00:00 -> (? left)

Report ssids_df.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [18]:
encuesta_df

,Marca temporal,¿Cúal es su lugar de residencia?,"En el caso de que haya marcado anteriormente que su lugar de residencia NO es Alcoi. ¿Podría especificar de dónde es? (país, provincia, municipio...)",¿Con cuántas personas viaja?,¿Con cuántas personas de las que viaja han entrado a las Tourist-Info?,Con el fin de mejorar nuestros servicios de información turística. ¿Me podría indicar si tiene el Wifi de su teléfono móvil abierto para conectarse a redes Wifi disponibles?
0,2021/06/22 10:05:40 a. m. EET,Alcoi,NaN,2,2,No entiendo la pregunta
1,2021/06/22 10:58:33 a. m. EET,España,Madrid,0,0,No
2,2021/06/23 10:13:42 a. m. EET,Alcoi,NaN,0,0,Sí
3,2021/06/23 10:14:04 a. m. EET,España,Amposta,1,1,Sí
4,2021/06/23 10:46:55 a. m. EET,Internacional,Belgica,1,1,Sí
...,...,...,...,...,...,...
2592,2022/02/28 5:11:25 p. m. EET,Internacional,Francia,4,4,Sí
2593,2022/02/28 5:46:35 p. m. EET,Comunidad Valenciana,Torreviaje,1,1,No
2594,2022/02/28 6:21:35 p. m. EET,Internacional,Chile,0,0,No
2595,2022/02/28 6:22:04 p. m. EET,Alcoi,NaN,2,2,Sí
